# A critical on the role of randomness-oriented components of computational intelligance-asbed optimization [Collective intelligance]


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from pathlib import Path

%matplotlib inline

In [2]:
def init_genrator(min_val, max_val, num_variables, pop_size):
    return np.random.uniform(min_val, max_val, (pop_size, num_variables))

def boundery_checker(pop, min_val, max_val):
    adj_pop = np.where(pop<min_val, min_val, pop)
    adj_pop = np.where(adj_pop>max_val, max_val, adj_pop)
    return adj_pop

def sorting_pop(pop, obj_func):
    results = obj_func(pop)
    indeces = np.argsort(results)
    return pop[indeces]

def collective_intelligance(pop, vect_lenght):
    rand = np.random.uniform(0, vect_lenght, size=(len(pop), 1))
    elite = pop[0]
    diff = elite - pop
    return pop + rand*diff
   
def pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val):
    NFE_value = 0
    result = np.zeros(iteration)
    NFE = np.zeros(iteration)
    pop = init_genrator(min_val, max_val, num_variables, pop_size)
    for i in range(iteration):
        pop_new = init_genrator(min_val, max_val, num_variables, pop_size)
        overall = np.concatenate((pop, pop_new), axis = 0)
        overall = sorting_pop(overall, obj_func)
        pop = overall[:pop_size]
        NFE_value += (pop_size*2)
        result[i] = obj_func([pop[0]])
        NFE[i] = NFE_value
    return pop[0], obj_func([pop[0]]), result, NFE
    
def random_greedy(pop_size, iteration, num_variables, obj_func, min_val, max_val):
    NFE_value = 0
    result = np.zeros(iteration)
    NFE = np.zeros(iteration)
    pop = init_genrator(min_val, max_val, num_variables, pop_size)
    for i in range(iteration):
        pop_new = init_genrator(min_val, max_val, num_variables, pop_size)
        of_vals = obj_func(pop)
        of_vals_new = obj_func(pop_new)
        pop = np.where((of_vals_new<of_vals).reshape(-1,1), pop_new, pop)
        NFE_value += (pop_size*2)
        pop = sorting_pop(pop, obj_func)
        result[i] = obj_func([pop[0]])
        NFE[i] = NFE_value
    return pop[0], obj_func([pop[0]]), result, NFE

def elite_guided_collective_intelligance(pop_size, iteration, num_variables, obj_func, min_val, max_val, vect_lenght):
    NFE_value = 0
    result = np.zeros(iteration)
    NFE = np.zeros(iteration)
    pop = init_genrator(min_val, max_val, num_variables, pop_size)
    pop = sorting_pop(pop, obj_func)
    NFE_value += pop_size
    for i in range(iteration):
        pop_new = collective_intelligance(pop, vect_lenght)
        overall = np.concatenate((pop, pop_new), axis = 0)
        overall = sorting_pop(overall, obj_func)
        pop = overall[:pop_size]
        NFE_value += (pop_size*2)
        result[i] = obj_func([pop[0]])
        NFE[i] = NFE_value
    return pop[0], obj_func([pop[0]]), result, NFE
               
def elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables, obj_func,
                                                min_val, max_val, vect_lenght):
    NFE_value = 0
    result = np.zeros(iteration)
    NFE = np.zeros(iteration)
    pop = init_genrator(min_val, max_val, num_variables, pop_size)
    pop = sorting_pop(pop, obj_func)
    NFE_value += pop_size
    for i in range(iteration):
        pop_new = collective_intelligance(pop, vect_lenght)
        of_vals = obj_func(pop)
        of_vals_new = obj_func(pop_new)
        pop = np.where((of_vals_new<of_vals).reshape(-1,1), pop_new, pop)
        pop = sorting_pop(pop, obj_func)
        NFE_value += (pop_size*3)
        result[i] = obj_func([pop[0]])
        NFE[i] = NFE_value
    return pop[0], obj_func([pop[0]]), result, NFE

<div class='alert alert-danger'>
    For Non-Centered Tests
</div>

In [ ]:
from cec2017.functions import *

<div class='alert alert-danger'>
    For Centered Tests
</div>

In [ ]:
def f1(x):
    x = np.array(x)
    x2 = x * x
    return np.sum(x2, axis=1)

def f2(x):
    x = np.array(x)
    ab = np.abs(x)
    a = np.sum(ab, axis=1)
    b = np.prod(ab, axis=1)
    return a + b

def f4(x):
    x = np.array(x)
    ab = np.abs(x)
    return np.max(ab, axis=1)

def f6(x):
    x = np.array(x)
    xnew = x + .5
    floor = np.floor(xnew)
    x2 = floor * floor
    return np.sum(x2, axis=1)

def f7(x):
    x = np.array(x)
    index = np.arange(1,len(x[0])+1)
    ones = np.ones_like(x)
    aa = ones * index
    x2 = x * x
    x4 = x2 * x2
    b = x4 * aa
    return np.sum(b, axis=1)

### I. Pure Random Optimization Algorithm

<div class='alert alert-info'>
    Non-Centered tests
</div>

In [ ]:
alg_name = 'Pure Random'
num_runs = 100

min_val, max_val = -100, 100
pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
funcs_index = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]

In [ ]:
for j in funcs_index:
    func_name = 'f%s'%j
    obj_func = eval(func_name)
    print(func_name)
    for i in range(len(pop_values_mesh.flat)):
        pop_size = pop_values_mesh.flat[i]
        iteration = iteration_values_mesh.flat[i]
        num_variables = num_variables_values_mesh.flat[i]
        overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                               'Iteration', 'NFE', 'time (s)', 'Best DVs'])
        for m in range(num_runs):
            start_time = time.time()
            X_best, OF_best, result, NFE = pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val)
            end_time = time.time()
            overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                    'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                    'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                    'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                   ignore_index=True)

        output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
        output_dir = Path('%s/Result/%s'%(alg_name, func_name))
        output_dir.mkdir(parents=True, exist_ok=True)
        overall_result.to_csv(output_dir / output_file)    

In [ ]:
for f in funcs_index:
    for n in num_variables_values:   
        summary_report = pd.DataFrame(columns=['Setting', 'NFE', 'Best outcome', 'Worst outcome', 'Avr. of outcomes',
                                               'Median of outcomes', 'Std of outcome', 'Avr. runtime (s)',
                                               'Fastest runtime (s)', 'Slowest runtime (s)'])
        for p in pop_values:
            for i in iteration_values:
                file_name = '%s/Result/f%d/%dvar_%dpop_%dit.csv'%(alg_name, f, n, p, i)
                test_report = pd.read_csv(file_name, index_col=[0])
                
                bst_out = test_report['best_OF'].min()
                wrst_out = test_report['best_OF'].max()
                avr_out = test_report['best_OF'].mean()
                median_out = test_report['best_OF'].median()
                std_out = test_report['best_OF'].std()
                avr_time = test_report['time (s)'].mean()
                fst_time = test_report['time (s)'].min()
                slwst_time = test_report['time (s)'].max()
                avr_NFE = test_report['NFE'].mean()
                
                summary_report = summary_report.append({'Setting': {'Pop': p, 'It.': i}, 'NFE': avr_NFE,
                                                        'Best outcome':bst_out, 'Worst outcome':wrst_out,
                                                        'Avr. of outcomes':avr_out, 'Median of outcomes':median_out,
                                                        'Std of outcome':std_out, 'Avr. runtime (s)':avr_time,
                                                        'Fastest runtime (s)':fst_time, 'Slowest runtime (s)':slwst_time},
                                                       ignore_index=True)
        output_file = '%dvar__summary.csv'%(n)
        output_dir = Path('%s/Summary Report/f%d'%(alg_name, f))
        output_dir.mkdir(parents=True, exist_ok=True)
        summary_report.to_csv(output_dir / output_file)

<div class='alert alert-info'>
    Centered test
</div>

In [ ]:
alg_name = 'Pure Random'
num_runs = 100

pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
range_dic = {1:(-100, 100), 2:(-10, 10), 4:(-100, 100), 6:(-100, 100), 7:(-1.28, 1.28)}
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
funcs_index = [1,2,4,6,7]

In [ ]:
for j in funcs_index:
    func_name = 'f%s'%j
    obj_func = eval(func_name)
    min_val, max_val = range_dic[j]
    print(func_name)
    for i in range(len(pop_values_mesh.flat)):
        pop_size = pop_values_mesh.flat[i]
        iteration = iteration_values_mesh.flat[i]
        num_variables = num_variables_values_mesh.flat[i]
        overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                               'Iteration', 'NFE', 'time (s)', 'Best DVs'])
        for m in range(num_runs):
            start_time = time.time()
            X_best, OF_best, result, NFE = pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val)
            end_time = time.time()
            overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                    'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                    'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                    'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                   ignore_index=True)

        output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
        output_dir = Path('%s/Result/Centered/%s'%(alg_name, func_name))
        output_dir.mkdir(parents=True, exist_ok=True)
        overall_result.to_csv(output_dir / output_file)    

## II. Pure Random with Greedy Strategy Optimization Algorithm

<div class='alert alert-info'>
    Non-Centered tests
</div>

In [ ]:
alg_name = 'Pure Random Greedy'
num_runs = 100

min_val, max_val = -100, 100
pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
funcs_index = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]

In [ ]:
for j in funcs_index:
    func_name = 'f%s'%j
    obj_func = eval(func_name)
    print(func_name)
    for i in range(len(pop_values_mesh.flat)):
        pop_size = pop_values_mesh.flat[i]
        iteration = iteration_values_mesh.flat[i]
        num_variables = num_variables_values_mesh.flat[i]
        overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                               'Iteration', 'NFE', 'time (s)', 'Best DVs'])
        for m in range(num_runs):
            start_time = time.time()
            X_best, OF_best, result, NFE = random_greedy(pop_size, iteration, num_variables, obj_func, min_val, max_val)
            end_time = time.time()
            overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                    'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                    'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                    'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                   ignore_index=True)

        output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
        output_dir = Path('%s/Result/%s'%(alg_name, func_name))
        output_dir.mkdir(parents=True, exist_ok=True)
        overall_result.to_csv(output_dir / output_file)

In [ ]:
funcs_index = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]
for f in funcs_index:
    for n in num_variables_values:   
        summary_report = pd.DataFrame(columns=['Setting', 'NFE', 'Best outcome', 'Worst outcome', 'Avr. of outcomes',
                                               'Median of outcomes', 'Std of outcome', 'Avr. runtime (s)',
                                               'Fastest runtime (s)', 'Slowest runtime (s)'])
        for p in pop_values:
            for i in iteration_values:
                file_name = '%s/Result/f%d/%dvar_%dpop_%dit.csv'%(alg_name, f, n, p, i)
                test_report = pd.read_csv(file_name, index_col=[0])
                
                bst_out = test_report['best_OF'].min()
                wrst_out = test_report['best_OF'].max()
                avr_out = test_report['best_OF'].mean()
                median_out = test_report['best_OF'].median()
                std_out = test_report['best_OF'].std()
                avr_time = test_report['time (s)'].mean()
                fst_time = test_report['time (s)'].min()
                slwst_time = test_report['time (s)'].max()
                avr_NFE = test_report['NFE'].mean()
                
                summary_report = summary_report.append({'Setting': {'Pop': p, 'It.': i}, 'NFE': avr_NFE,
                                                        'Best outcome':bst_out, 'Worst outcome':wrst_out,
                                                        'Avr. of outcomes':avr_out, 'Median of outcomes':median_out,
                                                        'Std of outcome':std_out, 'Avr. runtime (s)':avr_time,
                                                        'Fastest runtime (s)':fst_time, 'Slowest runtime (s)':slwst_time},
                                                       ignore_index=True)
        output_file = '%dvar__summary.csv'%(n)
        output_dir = Path('%s/Summary Report/f%d'%(alg_name, f))
        output_dir.mkdir(parents=True, exist_ok=True)
        summary_report.to_csv(output_dir / output_file)

<div class='alert alert-info'>
    Centered test
</div>

In [ ]:
alg_name = 'Pure Random Greedy'
num_runs = 100

pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
range_dic = {1:(-100, 100), 2:(-10, 10), 4:(-100, 100), 6:(-100, 100), 7:(-1.28, 1.28)}
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
funcs_index = [1,2,4,6,7]

In [ ]:
for j in funcs_index:
    func_name = 'f%s'%j
    obj_func = eval(func_name)
    min_val, max_val = range_dic[j]
    print(func_name)
    for i in range(len(pop_values_mesh.flat)):
        pop_size = pop_values_mesh.flat[i]
        iteration = iteration_values_mesh.flat[i]
        num_variables = num_variables_values_mesh.flat[i]
        overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                               'Iteration', 'NFE', 'time (s)', 'Best DVs'])
        for m in range(num_runs):
            start_time = time.time()
            X_best, OF_best, result, NFE = random_greedy(pop_size, iteration, num_variables, obj_func, min_val, max_val)
            end_time = time.time()
            overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                    'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                    'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                    'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                   ignore_index=True)

        output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
        output_dir = Path('%s/Result/Centered/%s'%(alg_name, func_name))
        output_dir.mkdir(parents=True, exist_ok=True)
        overall_result.to_csv(output_dir / output_file)

## III. Elite Guided Collective Intelligance Optimization Algorithm

<div class='alert alert-info'>
    Non-Centered tests
</div>

In [ ]:
alg_name = 'Elite Collective'
num_runs = 100

min_val, max_val = -100, 100
pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
vect_lenght_values = np.array([.5, 1])
funcs_index = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]

In [ ]:
for vect_lenght in vect_lenght_values:
    for j in funcs_index:
        func_name = 'f%s'%j
        obj_func = eval(func_name)
        print(func_name)
        for i in range(len(pop_values_mesh.flat)):
            pop_size = pop_values_mesh.flat[i]
            iteration = iteration_values_mesh.flat[i]
            num_variables = num_variables_values_mesh.flat[i]
            overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                                   'Iteration', 'NFE', 'time (s)', 'Best DVs'])
            count = 0
            while count < num_runs:
                try:
                    start_time = time.time()
                    X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                                        obj_func, min_val, max_val, vect_lenght)
                    end_time = time.time()
                    overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                            'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                            'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                            'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                           ignore_index=True)
                    count += 1
                except:
                    pass

            output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
            output_dir = Path('%s/Result/vec_%.1f/%s'%(alg_name, vect_lenght, func_name))
            output_dir.mkdir(parents=True, exist_ok=True)
            overall_result.to_csv(output_dir / output_file)

In [ ]:
for vect_lenght in vect_lenght_values:
    for f in funcs_index:
        for n in num_variables_values:   
            summary_report = pd.DataFrame(columns=['Setting',  'NFE', 'Best outcome', 'Worst outcome', 'Avr. of outcomes',
                                                   'Median of outcomes', 'Std of outcome', 'Avr. runtime (s)',
                                                   'Fastest runtime (s)', 'Slowest runtime (s)'])
            for p in pop_values:
                for i in iteration_values:
                    file_name = '%s/Result/vec_%.1f/f%d/%dvar_%dpop_%dit.csv'%(alg_name, vect_lenght, f, n, p, i)
                    test_report = pd.read_csv(file_name, index_col=[0])

                    bst_out = test_report['best_OF'].min()
                    wrst_out = test_report['best_OF'].max()
                    avr_out = test_report['best_OF'].mean()
                    median_out = test_report['best_OF'].median()
                    std_out = test_report['best_OF'].std()
                    avr_time = test_report['time (s)'].mean()
                    fst_time = test_report['time (s)'].min()
                    slwst_time = test_report['time (s)'].max()
                    avr_NFE = test_report['NFE'].mean()

                    summary_report = summary_report.append({'Setting': {'Pop': p, 'It.': i}, 'NFE': avr_NFE,
                                                            'Best outcome':bst_out, 'Worst outcome':wrst_out,
                                                            'Avr. of outcomes':avr_out, 'Median of outcomes':median_out,
                                                            'Std of outcome':std_out, 'Avr. runtime (s)':avr_time,
                                                            'Fastest runtime (s)':fst_time,
                                                            'Slowest runtime (s)':slwst_time},
                                                           ignore_index=True)
            output_file = '%dvar__summary.csv'%(n)
            output_dir = Path('%s/Summary Report/vec_%.2f/f%d'%(alg_name, vect_lenght, f))
            output_dir.mkdir(parents=True, exist_ok=True)
            summary_report.to_csv(output_dir / output_file)

<div class='alert alert-info'>
    Centered test
</div>

In [ ]:
alg_name = 'Elite Collective'
num_runs = 100

pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
range_dic = {1:(-100, 100), 2:(-10, 10), 4:(-100, 100), 6:(-100, 100), 7:(-1.28, 1.28)}
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
vect_lenght_values = np.array([.5, 1])
funcs_index = [1,2,4,6,7]

In [ ]:
for vect_lenght in vect_lenght_values:
    for j in funcs_index:
        func_name = 'f%s'%j
        obj_func = eval(func_name)
        min_val, max_val = range_dic[j]
        print(func_name)
        for i in range(len(pop_values_mesh.flat)):
            pop_size = pop_values_mesh.flat[i]
            iteration = iteration_values_mesh.flat[i]
            num_variables = num_variables_values_mesh.flat[i]
            overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                                   'Iteration', 'NFE', 'time (s)', 'Best DVs'])
            count = 0
            while count < num_runs:
                try:
                    start_time = time.time()
                    X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                                        obj_func, min_val, max_val, vect_lenght)
                    end_time = time.time()
                    overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                            'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                            'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                            'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                           ignore_index=True)
                    count += 1
                except:
                    pass

            output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
            output_dir = Path('%s/Result/Centered/vec_%.1f/%s'%(alg_name, vect_lenght, func_name))
            output_dir.mkdir(parents=True, exist_ok=True)
            overall_result.to_csv(output_dir / output_file)

## IV. Elite Guided Collective Intelligance with Greedy Strategy Optimization Algorithm

<div class='alert alert-info'>
    Non-Centered tests
</div>

In [ ]:
alg_name = 'Elite Greedy Collective'
num_runs = 100

min_val, max_val = -100, 100
pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
vect_lenght_values = np.array([.5, 1])
funcs_index = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]

In [ ]:
for vect_lenght in vect_lenght_values:
    for j in funcs_index:
        func_name = 'f%s'%j
        obj_func = eval(func_name)
        print(func_name)
        for i in range(len(pop_values_mesh.flat)):
            pop_size = pop_values_mesh.flat[i]
            iteration = iteration_values_mesh.flat[i]
            num_variables = num_variables_values_mesh.flat[i]
            overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                                   'Iteration', 'NFE', 'time (s)', 'Best DVs'])
            count = 0
            while count < num_runs:
                try:
                    start_time = time.time()
                    X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration,
                                                                                               num_variables,
                                                                                               obj_func, min_val,
                                                                                               max_val, vect_lenght)
                    end_time = time.time()
                    overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                            'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                            'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                            'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                           ignore_index=True)
                    count += 1
                    
                except:
                    pass

            output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
            output_dir = Path('%s/Result/vec_%.1f/%s'%(alg_name, vect_lenght, func_name))
            output_dir.mkdir(parents=True, exist_ok=True)
            overall_result.to_csv(output_dir / output_file)

In [ ]:
for vect_lenght in vect_lenght_values:
    for f in funcs_index:
        for n in num_variables_values:   
            summary_report = pd.DataFrame(columns=['Setting', 'NFE', 'Best outcome', 'Worst outcome', 'Avr. of outcomes',
                                                   'Median of outcomes', 'Std of outcome', 'Avr. runtime (s)',
                                                   'Fastest runtime (s)', 'Slowest runtime (s)'])
            for p in pop_values:
                for i in iteration_values:
                    file_name = '%s/Result/vec_%.1f/f%d/%dvar_%dpop_%dit.csv'%(alg_name, vect_lenght, f, n, p, i)
                    test_report = pd.read_csv(file_name, index_col=[0])

                    bst_out = test_report['best_OF'].min()
                    wrst_out = test_report['best_OF'].max()
                    avr_out = test_report['best_OF'].mean()
                    median_out = test_report['best_OF'].median()
                    std_out = test_report['best_OF'].std()
                    avr_time = test_report['time (s)'].mean()
                    fst_time = test_report['time (s)'].min()
                    slwst_time = test_report['time (s)'].max()
                    avr_NFE = test_report['NFE'].mean()

                    summary_report = summary_report.append({'Setting': {'Pop': p, 'It.': i}, 'NFE': avr_NFE,
                                                            'Best outcome':bst_out, 'Worst outcome':wrst_out,
                                                            'Avr. of outcomes':avr_out, 'Median of outcomes':median_out,
                                                            'Std of outcome':std_out, 'Avr. runtime (s)':avr_time,
                                                            'Fastest runtime (s)':fst_time,
                                                            'Slowest runtime (s)':slwst_time},
                                                           ignore_index=True)
            output_file = '%dvar__summary.csv'%(n)
            output_dir = Path('%s/Summary Report/vec_%.2f/f%d'%(alg_name, vect_lenght, f))
            output_dir.mkdir(parents=True, exist_ok=True)
            summary_report.to_csv(output_dir / output_file)

<div class='alert alert-info'>
    Centered test
</div>

In [ ]:
alg_name = 'Elite Greedy Collective'
num_runs = 100

min_val, max_val = -100, 100
pop_values = np.array([50, 100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])
range_dic = {1:(-100, 100), 2:(-10, 10), 4:(-100, 100), 6:(-100, 100), 7:(-1.28, 1.28)}
pop_values_mesh, iteration_values_mesh, num_variables_values_mesh = np.meshgrid(pop_values,
                                                                                iteration_values,
                                                                                num_variables_values)
vect_lenght_values = np.array([.5, 1])
funcs_index = [1,2,4,6,7]

In [ ]:
for vect_lenght in vect_lenght_values:
    for j in funcs_index:
        func_name = 'f%s'%j
        obj_func = eval(func_name)
        min_val, max_val = range_dic[j]
        print(func_name)
        for i in range(len(pop_values_mesh.flat)):
            pop_size = pop_values_mesh.flat[i]
            iteration = iteration_values_mesh.flat[i]
            num_variables = num_variables_values_mesh.flat[i]
            overall_result = pd.DataFrame(columns=['Function', 'Num Variables', 'best_OF', 'Pop Size',
                                                   'Iteration', 'NFE', 'time (s)', 'Best DVs'])
            count = 0
            while count < num_runs:
                try:
                    start_time = time.time()
                    X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration,
                                                                                               num_variables,
                                                                                               obj_func, min_val,
                                                                                               max_val, vect_lenght)
                    end_time = time.time()
                    overall_result = overall_result.append({'Function':func_name, 'Num Variables': num_variables,
                                                            'best_OF':OF_best[0], 'Pop Size': pop_size, 
                                                            'Iteration': iteration, 'NFE':NFE[-1].astype(int),
                                                            'time (s)':end_time-start_time, 'Best DVs': X_best},
                                                           ignore_index=True)
                    count += 1
                    
                except:
                    pass

            output_file = '%dvar_%dpop_%dit.csv'%(num_variables, pop_size, iteration)
            output_dir = Path('%s/Result/Centered/vec_%.1f/%s'%(alg_name, vect_lenght, func_name))
            output_dir.mkdir(parents=True, exist_ok=True)
            overall_result.to_csv(output_dir / output_file)

-------
# Individual Tests
-------

#### Pure Random Optimization Algorithm

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Pure Random'
min_val, max_val = -100, 100

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

#### Pure Random with Greedy Strategy Optimization Algorithm

In [ ]:
obj_func = f28
func_name = 'f3'
alg_name = 'Pure Random Greedy'
num_variables = 2
min_val, max_val = -100, 100

pop_size, iteration = 100, 200

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = random_greedy(pop_size, iteration, num_variables, obj_func, min_val, max_val)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

#### Elite Guided Collective Intelligance Optimization Algorithm

##### Vector Lenght for Elite Correction [vect_lenght = .5]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = .5

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                    obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 1]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 1

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                    obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 1.5]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 1.5

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                    obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 2]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 2

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                    obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 2.5]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 2.5

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                    obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

#### Vector Lenght for Elite Correction [vect_lenght = 3]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 3

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance(pop_size, iteration, num_variables,
                                                                    obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

#### Elite Guided Collective Intelligance with Greedy Strategy Optimization Algorithm

##### Vector Lenght for Elite Correction [vect_lenght = .5]

In [ ]:
obj_func = f21
func_name = 'f24'
alg_name = 'Elite Greedy Collective'
num_variables = 2
min_val, max_val = -50, 500

pop_size, iteration = 200, 500
vect_lenght = .5

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables,
                                                                           obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 1]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Greedy Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 1

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables,
                                                                           obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 1.5]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Greedy Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 1.5

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables,
                                                                           obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 2]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Greedy Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 2

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables,
                                                                           obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 2.5]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Greedy Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 2.5

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables,
                                                                           obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

##### Vector Lenght for Elite Correction [vect_lenght = 3]

In [ ]:
obj_func = f3
func_name = 'f3'
alg_name = 'Elite Greedy Collective'
num_variables = 10
min_val, max_val = -100, 100

pop_size, iteration = 1000, 5000
vect_lenght = 3

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = elite_guided_collective_intelligance_greedy(pop_size, iteration, num_variables,
                                                                           obj_func, min_val, max_val, vect_lenght)
end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()